In [1]:
# %%
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import datetime
from datetime import datetime as dt


print("🚀 Starting LFB Data Processing Pipeline...")
print("=" * 60)

# Define the column data types for consistent data loading
dtype_spec = {
    'IncidentNumber': 'string',
    'DateOfCall': 'object',
    'CalYear': 'Int16',
    'TimeOfCall': 'object',
    'HourOfCall': 'UInt8',
    'IncidentGroup': 'category',
    'StopCodeDescription': 'category',
    'SpecialServiceType': 'category',
    'PropertyCategory': 'category',
    'PropertyType': 'category',
    'AddressQualifier': 'category',
    'Postcode_full': 'string',
    'Postcode_district': 'string',
    'UPRN': 'Float64',
    'USRN': 'Int64',
    'IncGeo_BoroughCode': 'category',
    'IncGeo_BoroughName': 'category',
    'ProperCase': 'category',
    'IncGeo_WardCode': 'category',
    'IncGeo_WardName': 'category',
    'IncGeo_WardNameNew': 'category',
    'Easting_m': 'float64',
    'Northing_m': 'float64',
    'Easting_rounded': 'Int32',
    'Northing_rounded': 'Int32',
    'Latitude': 'float64',
    'Longitude': 'float64',
    'FRS': 'category',
    'IncidentStationGround': 'category',
    'FirstPumpArriving_AttendanceTime': 'Int16',
    'FirstPumpArriving_DeployedFromStation': 'category',
    'SecondPumpArriving_AttendanceTime': 'Int16',
    'SecondPumpArriving_DeployedFromStation': 'category',
    'NumStationsWithPumpsAttending': 'UInt8',
    'NumPumpsAttending': 'UInt8',
    'PumpCount': 'UInt16',
    'PumpMinutesRounded': 'UInt32',
    'Notional Cost (£)': 'UInt32',
    'NumCalls': 'UInt16'
}

print("📁 STEP 1: Loading Incident Data Files...")
print("-" * 40)

f_dir = input().strip()

# Read CSV file (2009-2017 data)
print("📊 Loading CSV file: LFB Incident data from 2009 - 2017.csv")
csv_file_path = f_dir + "\\LFB Incident data from 2009 - 2017.csv"
df_csv = pd.read_csv(
    csv_file_path,
    dtype=dtype_spec
)
print(f"   ✅ CSV loaded: {len(df_csv):,} records, {len(df_csv.columns)} columns")

# Process date and time columns for CSV data
print("   ⏰ Processing date/time columns for CSV data...")
df_csv['DateOfCall'] = pd.to_datetime(df_csv['DateOfCall'], format='mixed').dt.date
df_csv['DateOfCall'] = pd.to_datetime(df_csv['DateOfCall'])
df_csv['TimeOfCall'] = pd.to_timedelta(df_csv['TimeOfCall'].astype(str))
print("   ✅ Date/Time processing completed for CSV data")

# Read first XLSX file (2018-2023 data)
print("📊 Loading XLSX file: LFB Incident data from 2018 - 2023.xlsx")
xlsx_file1_path = f_dir + "\\LFB Incident data from 2018 - 2023.xlsx"
df_xlsx1 = pd.read_excel(
    xlsx_file1_path,
    dtype=dtype_spec
)
print(f"   ✅ XLSX 2018-2023 loaded: {len(df_xlsx1):,} records, {len(df_xlsx1.columns)} columns")

# Process date and time columns for first XLSX data
print("   ⏰ Processing date/time columns for 2018-2023 data...")
df_xlsx1['DateOfCall'] = pd.to_datetime(df_xlsx1['DateOfCall'], format='mixed').dt.date
df_xlsx1['DateOfCall'] = pd.to_datetime(df_xlsx1['DateOfCall'])
df_xlsx1['TimeOfCall'] = pd.to_timedelta(df_xlsx1['TimeOfCall'].astype(str))
print("   ✅ Date/Time processing completed for 2018-2023 data")

# Read second XLSX file (2024 onwards data)
print("📊 Loading XLSX file: LFB Incident data from 2024 onwards.xlsx")
xlsx_file2_path = f_dir + "\\LFB Incident data from 2024 onwards.xlsx"
df_xlsx2 = pd.read_excel(
    xlsx_file2_path,
    dtype=dtype_spec
)
print(f"   ✅ XLSX 2024+ loaded: {len(df_xlsx2):,} records, {len(df_xlsx2.columns)} columns")

# Process date and time columns for second XLSX data
print("   ⏰ Processing date/time columns for 2024+ data...")
df_xlsx2['DateOfCall'] = pd.to_datetime(df_xlsx2['DateOfCall'], format='mixed').dt.date
df_xlsx2['DateOfCall'] = pd.to_datetime(df_xlsx2['DateOfCall'])
df_xlsx2['TimeOfCall'] = pd.to_timedelta(df_xlsx2['TimeOfCall'].astype(str))
print("   ✅ Date/Time processing completed for 2024+ data")

print("🔄 STEP 2: Combining Incident Data...")
print("-" * 40)

# Combine all incident dataframes into one master dataframe
print("🔗 Concatenating all incident dataframes...")
LFB_Inc = pd.concat([df_csv, df_xlsx1, df_xlsx2], ignore_index=True)
print(f"   ✅ Combined incident data: {len(LFB_Inc):,} total records")

# Convert specific columns to category dtype for memory optimization and performance
print("🏷️  Converting text columns to category dtype for optimization...")
category_columns = [
    'StopCodeDescription', 'SpecialServiceType', 'PropertyType',
    'IncGeo_WardCode', 'IncGeo_WardName', 'IncGeo_WardNameNew',
    'IncidentStationGround', 'FirstPumpArriving_DeployedFromStation',
    'SecondPumpArriving_DeployedFromStation'
]

for col in category_columns:
    if col in LFB_Inc.columns:
        if col == 'PropertyType':  # Special handling for PropertyType
            LFB_Inc[col] = LFB_Inc[col].str.strip().astype('category')
        else:
            LFB_Inc[col] = LFB_Inc[col].astype('category')
        print(f"   ✅ Converted {col} to category")

print(f"🎉 Incident data processing completed! Final size: {len(LFB_Inc):,} records")

print("\n" + "=" * 60)
print("📁 STEP 3: Loading Mobilisation Data Files...")
print("-" * 40)

# Load mobilisation data from multiple files
print("📊 Loading mobilisation data files...")
df_2009_2014 = pd.read_excel(f_dir + "\\LFB Mobilisation data from January 2009 - 2014.xlsx")
df_2015_2020 = pd.read_excel(f_dir + "\\LFB Mobilisation data from 2015 - 2020.xlsx")
df_2021_2024 = pd.read_csv(f_dir + "\\LFB Mobilisation data from 2021 - 2024.csv")
df_2025 = pd.read_csv(f_dir + "\\LFB Mobilisation data from 2025.csv")

print(f"   ✅ 2009-2014: {len(df_2009_2014):,} records")
print(f"   ✅ 2015-2020: {len(df_2015_2020):,} records")
print(f"   ✅ 2021-2024: {len(df_2021_2024):,} records")
print(f"   ✅ 2025: {len(df_2025):,} records")

def convert_dtypes(df, df_name=""):
    """
    Convert data types for LFB dataset columns to optimize memory usage and ensure consistency

    Parameters:
    df (pd.DataFrame): DataFrame to modify
    df_name (str): Optional name for logging/debugging

    Returns:
    pd.DataFrame: Modified DataFrame with converted dtypes
    """
    if df_name:
        print(f"   🔧 Converting dtypes for {df_name}...")

    # Define the conversion mapping for mobilisation data
    dtype_conversions = {
        'IncidentNumber': 'string',
        'CalYear': 'Int16',
        'HourOfCall': 'UInt8',
        'ResourceMobilisationId': 'string',
        'Resource_Code': 'category',
        'PerformanceReporting': 'category',
        'TurnoutTimeSeconds': 'Int16',
        'TravelTimeSeconds': 'Int16',
        'AttendanceTimeSeconds': 'Int16',
        'DeployedFromStation_Code': 'category',
        'DeployedFromStation_Name': 'category',
        'DeployedFromLocation': 'category',
        'PumpOrder': 'category',
        'PlusCode_Code': 'category',
        'PlusCode_Description': 'category',
        'DelayCodeId': 'category',
        'DelayCode_Description': 'category'
    }

    # Apply conversions only for columns that exist in the dataframe
    conversion_count = 0
    for column, dtype in dtype_conversions.items():
        if column in df.columns:
            df[column] = df[column].astype(dtype)
            conversion_count += 1
        elif df_name:
            print(f"   ⚠️  Warning: Column '{column}' not found in {df_name}")

    if df_name:
        print(f"   ✅ Converted {conversion_count} columns in {df_name}")

    return df

print("🔄 STEP 4: Processing Mobilisation Data...")
print("-" * 40)

# Create dictionary of dataframes for batch processing
dataframes = {
    '2009-2014': df_2009_2014,
    '2015-2020': df_2015_2020,
    '2021-2024': df_2021_2024,
    '2025': df_2025
}

# Apply dtype conversion to all mobilisation dataframes
for name, df in dataframes.items():
    dataframes[name] = convert_dtypes(df, f"df_{name}")

# Update the original dataframe variables
df_2009_2014, df_2015_2020, df_2021_2024, df_2025 = dataframes.values()

print("🗑️  Removing unnecessary columns from recent data...")
# Remove redundant columns from 2021-2024 and 2025 data
columns_to_drop = ['BoroughName', 'WardName']
for col in columns_to_drop:
    if col in df_2021_2024.columns:
        df_2021_2024 = df_2021_2024.drop(columns=col)
        print(f"   ✅ Dropped {col} from 2021-2024 data")

for col in columns_to_drop:
    if col in df_2025.columns:
        df_2025 = df_2025.drop(columns=col)
        print(f"   ✅ Dropped {col} from 2025 data")

print("⏰ Processing datetime columns for mobilisation data...")
# Process datetime columns for mobilization data

mobilization_dfs = [df_2009_2014, df_2015_2020, df_2021_2024, df_2025]

for i in mobilization_dfs:
    i['DateAndTimeMobilised'] = pd.to_datetime(i['DateAndTimeMobilised'], format='mixed')
    i['DateMobilised'] = pd.to_datetime(i['DateAndTimeMobilised'], format='mixed').dt.date
    i['DateMobilised'] = pd.to_datetime(i['DateMobilised'])
    i['TimeMobilised'] = pd.to_datetime(i['DateAndTimeMobilised'], format='mixed').dt.time
    i['TimeMobilised'] = pd.to_timedelta(i['TimeMobilised'].astype(str))
    i['DateAndTimeMobile'] = pd.to_datetime(i['DateAndTimeMobile'], format='mixed')
    i['DateMobile'] = pd.to_datetime(i['DateAndTimeMobile'], format='mixed').dt.date
    i['DateMobile'] = pd.to_datetime(i['DateMobile'])
    i['TimeMobile'] = pd.to_datetime(i['DateAndTimeMobile'], format='mixed').dt.time
    i['TimeMobile'] = pd.to_timedelta(i['TimeMobile'].astype(str))
    i['DateAndTimeArrived'] = pd.to_datetime(i['DateAndTimeArrived'], format='mixed')
    i['DateArrived'] = pd.to_datetime(i['DateAndTimeArrived'], format='mixed').dt.date
    i['DateArrived'] = pd.to_datetime(i['DateArrived'])
    i['TimeArrived'] = pd.to_datetime(i['DateAndTimeArrived'], format='mixed').dt.time
    i['TimeArrived'] = pd.to_timedelta(i['TimeArrived'].astype(str))
    i['DateAndTimeLeft'] = pd.to_datetime(i['DateAndTimeLeft'], format='mixed')
    i['DateLeft'] = pd.to_datetime(i['DateAndTimeLeft'], format='mixed').dt.date
    i['DateLeft'] = pd.to_datetime(i['DateLeft'])
    i['TimeLeft'] = pd.to_datetime(i['DateAndTimeLeft'], format='mixed').dt.time
    i['TimeLeft'] = pd.to_timedelta(i['TimeLeft'].astype(str))
    i['DateAndTimeReturned'] = pd.to_datetime(i['DateAndTimeReturned'], format='mixed')
    i['DateReturned'] = pd.to_datetime(i['DateAndTimeReturned'], format='mixed').dt.date
    i['DateReturned'] = pd.to_datetime(i['DateReturned'])
    i['TimeReturned'] = pd.to_datetime(i['DateAndTimeReturned'], format='mixed').dt.time
    i['TimeReturned'] = pd.to_timedelta(i['TimeReturned'].astype(str))

print("🔗 Combining all mobilisation dataframes...")
# Combine all mobilisation dataframes
LFB_Mob = pd.concat(
    [df_2009_2014, df_2015_2020, df_2021_2024, df_2025],
    ignore_index=True,
    sort=False
)
LFB_Mob.drop_duplicates(inplace=True)

x = pd.to_numeric(LFB_Mob['PerformanceReporting'], errors='coerce')
LFB_Mob['PerformanceReporting'] = np.where(x.isna(), LFB_Mob['PerformanceReporting'], x.astype('UInt8'))
LFB_Mob['PerformanceReporting'] = LFB_Mob['PerformanceReporting'].astype('category')

print(f"   ✅ Combined mobilisation data: {len(LFB_Mob):,} total records")

# Apply final dtype conversion and process datetime columns
print("⏰ Processing datetime columns for mobilisation data...")
LFB_Mob = convert_dtypes(LFB_Mob, "LFB_Mob (final)")
LFB_Mob['DateAndTimeMobilised'] = pd.to_datetime(LFB_Mob['DateAndTimeMobilised'])
LFB_Mob['DateAndTimeMobile'] = pd.to_datetime(LFB_Mob['DateAndTimeMobile'])
LFB_Mob['DateAndTimeArrived'] = pd.to_datetime(LFB_Mob['DateAndTimeArrived'])
LFB_Mob['DateAndTimeLeft'] = pd.to_datetime(LFB_Mob['DateAndTimeLeft'])
LFB_Mob['DateAndTimeReturned'] = pd.to_datetime(LFB_Mob['DateAndTimeReturned'])

print("   ✅ Datetime processing completed for mobilisation data")

print(f"🎉 Mobilisation data processing completed! Final size: {len(LFB_Mob):,} records")

print("\n" + "=" * 60)
print("\n" + "=" * 60)
print("📊 STEP 5: Creating Data Type Documentation...")
print("-" * 40)

print("📋 Gathering data type information from both datasets...")
# Create data type documentation for both datasets
dtypes_LFB_Inc = pd.DataFrame({
    'Column': LFB_Inc.columns,
    'Dtype': LFB_Inc.dtypes.values,
    'Source': 'LFB_Inc'
})

dtypes_LFB_Mob = pd.DataFrame({
    'Column': LFB_Mob.columns,
    'Dtype': LFB_Mob.dtypes.values,
    'Source': 'LFB_Mob'
})

print("🔗 Combining data type information...")
LFB_stacked_dtypes = pd.concat([dtypes_LFB_Inc, dtypes_LFB_Mob], ignore_index=True)
print(f"   ✅ Combined dtype info: {len(LFB_stacked_dtypes)} columns documented")

print("\n" + "=" * 60)
print("📊 STEP 6: Creating unique and NA Data Documentation...")
print("-" * 40)

print("📋 Gathering unique values and empty cells (isna) data information from both datasets...")
# Create unique values and empty cells (isna) data documentation for both datasets
LFB_Inc_cal = pd.concat([LFB_Inc.notna().sum(),LFB_Inc.isna().sum(),LFB_Inc.nunique()],axis=1)
LFB_Inc_cal = LFB_Inc_cal.reset_index()
LFB_Inc_cal.columns = ['Column', 'Not_NA_Count', 'NA_Count','unique_Count']

LFB_Mob_cal = pd.concat([LFB_Mob.notna().sum(),LFB_Mob.isna().sum(),LFB_Mob.nunique()],axis=1)
LFB_Mob_cal = LFB_Mob_cal.reset_index()
LFB_Mob_cal.columns = ['Column', 'Not_NA_Count', 'NA_Count','unique_Count']

print("🔗 Combining unique and empty cells (isna) data information...")
LFB_cal = pd.concat([LFB_Inc_cal, LFB_Mob_cal], ignore_index=True)
print(f"   ✅ Combined unique and isna info: {len(LFB_cal)} columns documented")

print("📖 Loading and combining metadata files...")
# Load and combine metadata from Excel files
LFB_Inc_md = pd.read_excel(f_dir + "\\Metadata.xlsx")
LFB_Mob_md = pd.read_excel(f_dir + "\\Mobilisations Metadata.xlsx")
LFB_md = pd.concat([LFB_Inc_md, LFB_Mob_md], ignore_index=True)

print("🔍 Merging metadata with data types and NA & unique value counts...")
# Merge metadata with data types,NA and unique values
LFB_md_dt = pd.merge(pd.merge(LFB_md,LFB_stacked_dtypes,on='Column'),LFB_cal,on='Column')
print("💾 Saving final metadata documentation...")
# Save the comprehensive metadata documentation
LFB_md_dt.to_excel('LFB_md_dt.xlsx', index=False)
print("   ✅ Metadata saved to 'LFB_md_dt.xlsx'")

print("\n" + "=" * 60)
print("🎉 PROCESSING COMPLETED SUCCESSFULLY!")
print("=" * 60)
print(f"📈 FINAL DATASET SUMMARY:")
print(f"   • LFB_Inc: {len(LFB_Inc):,} records")
print(f"   • LFB_Mob: {len(LFB_Mob):,} records")
print(f"   • Total documented columns: {len(LFB_stacked_dtypes)}")
print(f"   • Metadata file: LFB_md_dt.xlsx")
print("=" * 60)

df = LFB_Inc.copy()
df['Postcode_full'] = df['Postcode_full'].fillna('SW17 0QT')
categorical_columns = df.select_dtypes(include=['category']).columns

def fill_with_mode(series):
    mode_val = series.mode()
    if not mode_val.empty:
        return series.fillna(mode_val[0])
    return series

df[categorical_columns] = df[categorical_columns].apply(fill_with_mode)

numerical_cols = df.select_dtypes(include=[np.number]).columns
df[numerical_cols] = df[numerical_cols].fillna(0)
# Fixed data preprocessing function
def preprocess_data(df):
    # Make a copy to avoid modifying the original
    df_processed = df.copy()

    # Convert date columns if needed
    if 'DateOfCall' in df_processed.columns:
        df_processed['DateOfCall'] = pd.to_datetime(df_processed['DateOfCall'], errors='coerce')
        df_processed['Month'] = df_processed['DateOfCall'].dt.month
        df_processed['DayOfWeek'] = df_processed['DateOfCall'].dt.day_name()

    # Handle categorical columns properly
    categorical_columns = ['IncidentGroup', 'StopCodeDescription', 'IncGeo_BoroughName']

    for col in categorical_columns:
        if col in df_processed.columns:
            # Convert to string first, then handle missing values
            df_processed[col] = df_processed[col].astype(str)
            df_processed[col] = df_processed[col].replace('nan', 'Unknown')
            df_processed[col] = df_processed[col].fillna('Unknown')
            # Convert back to categorical if it was categorical
            if hasattr(df[col], 'cat'):
                df_processed[col] = pd.Categorical(df_processed[col])

    return df_processed

# Preprocess your data
df_processed = preprocess_data(df)

# Get unique values for filters
available_years = sorted(df_processed['CalYear'].unique())
boroughs = sorted(df_processed['IncGeo_BoroughName'].unique())
incident_groups = sorted(df_processed['IncidentGroup'].unique())

# Remove 'Unknown' from filter options if present
boroughs = [b for b in boroughs if b != 'Unknown']
incident_groups = [ig for ig in incident_groups if ig != 'Unknown']
df2 = LFB_Mob.copy()
df2['DateAndTimeMobile'] = df2['DateAndTimeMobile'].fillna(df2['DateAndTimeMobilised'])
df2['DateAndTimeMobilised'] = pd.to_datetime(df2['DateAndTimeMobilised'])
df2['TurnoutTimeSeconds'] = df2['TurnoutTimeSeconds'].fillna(df2['TurnoutTimeSeconds'].mean().round())
df2['TravelTimeSeconds'] = df2['TravelTimeSeconds'].fillna(df2['TravelTimeSeconds'].mean().round())
df2['DateAndTimeLeft'] = df2['DateAndTimeLeft'].fillna(df2['DateAndTimeArrived'])
df2['DateAndTimeLeft'] = pd.to_datetime(df2['DateAndTimeLeft'])
df2['DateAndTimeReturned'] = df2['DateAndTimeReturned'].fillna(df2['DateAndTimeLeft'])
df2['DateAndTimeReturned'] = pd.to_datetime(df2['DateAndTimeReturned'])
categorical_columns2 = df2.select_dtypes(include=['category']).columns
df2[categorical_columns2] = df2[categorical_columns2].apply(fill_with_mode)
df2['DateMobile'] = df2['DateMobile'].fillna(df2['DateMobilised'])
df2['TimeMobile'] = df2['TimeMobile'].fillna(df2['TimeMobilised'])
df2['DateLeft'] = df2['DateLeft'].fillna(df2['DateArrived'])
df2['TimeLeft'] = df2['TimeLeft'].fillna(df2['TimeArrived'])
df2['DateReturned'] = df2['DateReturned'].fillna(df2['DateLeft'])
df2['TimeReturned'] = df2['TimeReturned'].fillna(df2['TimeLeft'])

def preprocess_data2(df):
    # Make a copy to avoid modifying the original
    df_processed = df.copy()

    # Convert date columns if needed
    if 'DateOfCall' in df_processed.columns:
        df_processed['DateAndTimeMobilised'] = pd.to_datetime(df_processed['DateAndTimeMobilised'], errors='coerce')
        df_processed['Month'] = df_processed['DateOfCall'].dt.month
        df_processed['DayOfWeek'] = df_processed['DateOfCall'].dt.day_name()

    # Handle categorical columns properly
    categorical_columns = df.select_dtypes(include=['category']).columns

    for col in categorical_columns:
        if col in df_processed.columns:
            # Convert to string first, then handle missing values
            df_processed[col] = df_processed[col].astype(str)
            df_processed[col] = df_processed[col].replace('nan', 'Unknown')
            df_processed[col] = df_processed[col].fillna('Unknown')
            # Convert back to categorical if it was categorical
            if hasattr(df[col], 'cat'):
                df_processed[col] = pd.Categorical(df_processed[col])

    return df_processed

df_processed2 = preprocess_data2(df2)

🚀 Starting LFB Data Processing Pipeline...
📁 STEP 1: Loading Incident Data Files...
----------------------------------------
📊 Loading CSV file: LFB Incident data from 2009 - 2017.csv
   ✅ CSV loaded: 988,279 records, 39 columns
   ⏰ Processing date/time columns for CSV data...
   ✅ Date/Time processing completed for CSV data
📊 Loading XLSX file: LFB Incident data from 2018 - 2023.xlsx
   ✅ XLSX 2018-2023 loaded: 670,635 records, 39 columns
   ⏰ Processing date/time columns for 2018-2023 data...
   ✅ Date/Time processing completed for 2018-2023 data
📊 Loading XLSX file: LFB Incident data from 2024 onwards.xlsx
   ✅ XLSX 2024+ loaded: 236,763 records, 39 columns
   ⏰ Processing date/time columns for 2024+ data...
   ✅ Date/Time processing completed for 2024+ data
🔄 STEP 2: Combining Incident Data...
----------------------------------------
🔗 Concatenating all incident dataframes...
   ✅ Combined incident data: 1,895,677 total records
🏷️  Converting text columns to category dtype for opt

In [2]:
grouped_multiple = df_processed.groupby(['CalYear','IncidentGroup'],observed=False).agg({'IncidentNumber':['count']})
grouped_multiple = grouped_multiple.reset_index()
grouped_multiple.columns = ['Year', 'Incident type', 'Incident number']
fig = px.line(grouped_multiple, x="Year", y="Incident number", color="Incident type", title="incident counts by Year and Incident Group")
fig.update_xaxes(tickmode='linear',showticklabels=True)
fig.show()

In [3]:
grouped_multiple2 = df_processed.groupby(['HourOfCall','IncidentGroup'],observed=False).agg({'IncidentNumber':['count']})
grouped_multiple2 = grouped_multiple2.reset_index()
grouped_multiple2.columns = ['Hour', 'Incident type', 'Incident number']
fig = px.bar(grouped_multiple2, x="Hour", y="Incident number", color="Incident type", title="incident counts by Year and Incident Group",text_auto='.2s')
fig.update_xaxes(tickmode='linear',showticklabels=True)
fig.show()

In [4]:
grouped_multiple3 = df_processed.groupby(['CalYear','IncidentGroup'],observed=False).agg({'FirstPumpArriving_AttendanceTime':['mean']})
grouped_multiple3 = grouped_multiple3.reset_index()
grouped_multiple3.columns = ['Year','Incident type', 'Average Attendance Time (seconds)']
grouped_multiple3['Average Attendance Time (seconds)'] = grouped_multiple3['Average Attendance Time (seconds)'].round().astype(int)
fig = px.bar(grouped_multiple3, x="Year", y="Average Attendance Time (seconds)", color="Incident type", title="Average Attendance Time (seconds) by Year and Incident Group",text_auto='.2s')
fig.update_xaxes(tickmode='linear',showticklabels=True)
fig.show()

In [5]:
incident_type_data = df_processed['IncidentGroup'].value_counts()
incident_type_fig = px.pie(values=incident_type_data.values,names=incident_type_data.index,color=incident_type_data.index,color_discrete_map={'Fire': '#dc3545','Special Service': '#28a745','False Alarm': '#ffc107'},title='Incident Type Distribution')
incident_type_fig.show()

In [6]:
borough_data = df_processed['ProperCase'].value_counts().head(10)
borough_fig = px.bar(x=borough_data.values,y=borough_data.index,orientation='h',color=borough_data.values,color_continuous_scale='Blues',title='Top 10 Boroughs by Number of Incidents',text_auto='.2s')
borough_fig.update_layout(xaxis_title='Number of Incidents',yaxis_title='Borough',showlegend=False,margin=dict(t=30, b=0, l=0, r=0))
borough_fig.show()

In [7]:
grouped_multiple3

,Year,Incident type,Average Attendance Time (seconds)
0,2009,False Alarm,295
1,2009,Fire,313
2,2009,Special Service,280
3,2010,False Alarm,297
4,2010,Fire,315
5,2010,Special Service,248
6,2011,False Alarm,293
7,2011,Fire,310
8,2011,Special Service,243
9,2012,False Alarm,292


In [8]:
df_processed.columns

Index(['IncidentNumber', 'DateOfCall', 'CalYear', 'TimeOfCall', 'HourOfCall',
       'IncidentGroup', 'StopCodeDescription', 'SpecialServiceType',
       'PropertyCategory', 'PropertyType', 'AddressQualifier', 'Postcode_full',
       'Postcode_district', 'UPRN', 'USRN', 'IncGeo_BoroughCode',
       'IncGeo_BoroughName', 'ProperCase', 'IncGeo_WardCode',
       'IncGeo_WardName', 'IncGeo_WardNameNew', 'Easting_m', 'Northing_m',
       'Easting_rounded', 'Northing_rounded', 'Latitude', 'Longitude', 'FRS',
       'IncidentStationGround', 'FirstPumpArriving_AttendanceTime',
       'FirstPumpArriving_DeployedFromStation',
       'SecondPumpArriving_AttendanceTime',
       'SecondPumpArriving_DeployedFromStation',
       'NumStationsWithPumpsAttending', 'NumPumpsAttending', 'PumpCount',
       'PumpMinutesRounded', 'Notional Cost (£)', 'NumCalls', 'Month',
       'DayOfWeek'],
      dtype='object')

In [9]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

# Group and prepare the data
grouped_multiple = df_processed.groupby(['CalYear','IncidentGroup'], observed=False).agg({'IncidentNumber': ['count']})
grouped_multiple = grouped_multiple.reset_index()
grouped_multiple.columns = ['Year', 'Incident type', 'Incident number']

# Get unique incident types and years
incident_types = sorted(grouped_multiple['Incident type'].unique())
years = sorted(grouped_multiple['Year'].unique())

# Create a color palette - using Plotly's qualitative color scale
colors = px.colors.qualitative.Set3  # You can change this to other palettes:
# Options: Set1, Set2, Set3, Pastel1, Pastel2, Dark2, Bold, Vivid, etc.

# Create a color mapping for incident types
color_map = {incident_type: colors[i % len(colors)] 
             for i, incident_type in enumerate(incident_types)}

# Create figure
fig = go.Figure()

# Create dropdown menu options
dropdown_options = []

# Pivot data for "All Years" - sum across all years
all_data_pivot = grouped_multiple.groupby('Incident type')['Incident number'].sum().reset_index()
all_data_pivot = all_data_pivot.set_index('Incident type').reindex(incident_types).reset_index()
all_data_pivot['Incident number'] = all_data_pivot['Incident number'].fillna(0)

# Add "All Years" trace (this will be a single bar per incident type, so not stacked)
fig.add_trace(go.Bar(
    x=all_data_pivot['Incident type'],
    y=all_data_pivot['Incident number'],
    visible=True,
    text=all_data_pivot['Incident number'],
    textposition='auto',
    marker_color=[color_map[inc_type] for inc_type in all_data_pivot['Incident type']],
    name='All Years'
))

dropdown_options.append({'label': 'All Years', 'method': 'update', 'args': [
    {'visible': [True] + [False] * len(years)},
    {'title': 'Incident Counts - All Years'}
]})

# Prepare data for each year - create stacked bar chart traces
for i, year in enumerate(years):
    year_data = grouped_multiple[grouped_multiple['Year'] == year]
    
    # Create a complete DataFrame with all incident types for this year
    year_data_complete = pd.DataFrame({'Incident type': incident_types})
    year_data_complete = year_data_complete.merge(
        year_data, on='Incident type', how='left'
    )
    year_data_complete['Incident number'] = year_data_complete['Incident number'].fillna(0)
    
    # Add trace for this year (single trace per year for stacked view)
    fig.add_trace(go.Bar(
        x=year_data_complete['Incident type'],
        y=year_data_complete['Incident number'],
        visible=False,
        text=year_data_complete['Incident number'],
        textposition='auto',
        marker_color=[color_map[inc_type] for inc_type in year_data_complete['Incident type']],
        name=str(year)
    ))
    
    # Create visibility list for this year
    visibility = [False] * (len(years) + 1)
    visibility[i + 1] = True
    
    dropdown_options.append({'label': str(year), 'method': 'update', 'args': [
        {'visible': visibility},
        {'title': f'Incident Counts - {year}'}
    ]})

# Update layout for stacked bar chart
fig.update_layout(
    title='Incident Counts by Year and Incident Group',
    xaxis_title='Incident Type',
    yaxis_title='Incident Count',
    barmode='stack',  # This makes it a stacked bar chart
    updatemenus=[dict(
        buttons=dropdown_options,
        direction='down',
        showactive=True,
        x=0.1,
        xanchor='left',
        y=1.15,
        yanchor='top'
    )],
    legend_title="Incident Types",
    showlegend=True
)

# Update xaxis to show all incident types
fig.update_xaxes(
    categoryorder='array',
    categoryarray=incident_types
)

fig.show()

C:\Users\mohamhussein\AppData\Local\Temp\ipykernel_25772\1556728998.py:29: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

